# Analisis exploratorio de los distintos archivos 

* Total_koha: Este archivo contiene los datos pertenecientes a al sistema implementado en la Facultad de Ingeniera 
* Consulta_koha: Contiene los libros pertenecientes al sistema de la Facultad de Ingenieria marcados como material de consulta
* Bidi: Servicio de libros digitales en el que la facultad se encuetra suscrito

## Librerias de utilidad y ruta de acceso

In [8]:
import sys
import os

sys.path.append(os.path.abspath('C:/Users/Axel/Desktop/Practica-Biblioteca'))

from src import lectura, conversor
import pandas as pd



## Analisis de libros total koha

In [9]:
libros_total_koha = lectura.leer_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/total_koha.csv')

Archivo C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/total_koha.csv leído con éxito.


In [10]:
libros_total_koha.head()

,barcode,dateaccessioned,booksellerid,homebranch,price,replacementprice,itemcallnumber,holdingbranch,title,author,isbn
0,7230,2024-04-22,NaN,FING,NaN,NaN,32,FING,Chaos :,"Alligood, Kathleen T.",0387946772
1,7231,2024-04-22,NaN,FING,NaN,NaN,32,FING,Analysis of observed chaotic data /,"Abarbanel, H. D. I",0387983724
2,7229,2024-02-26,NaN,FING,NaN,NaN,50,FING,Proactive innovation :,"Richter, Sergio Raúl",9798861825221
3,L578,2023-11-08,NaN,FING,NaN,NaN,80,FING,GMP qualification and validation of facilities...,NaN,NaN
4,7222,2023-10-02,NaN,FING,NaN,NaN,49,FING,Introducción a la economía de la salud para in...,"Martínez Licona, Fabiola Margarita",9786072813410


In [11]:
libros_total_koha.info() #informacion de los datos


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4326 entries, 0 to 4325
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   barcode           4326 non-null   object 
 1   dateaccessioned   4326 non-null   object 
 2   booksellerid      0 non-null      float64
 3   homebranch        4326 non-null   object 
 4   price             103 non-null    float64
 5   replacementprice  1 non-null      float64
 6   itemcallnumber    4311 non-null   object 
 7   holdingbranch     4326 non-null   object 
 8   title             4326 non-null   object 
 9   author            3857 non-null   object 
 10  isbn              3413 non-null   object 
dtypes: float64(3), object(8)
memory usage: 371.9+ KB


In [12]:
# Mostrar un resumen descriptivo de las variables cualitativas
print(libros_total_koha.describe(include='object'))

       barcode dateaccessioned homebranch itemcallnumber holdingbranch  \
count     4326            4326       4326           4311          4326   
unique    4326              84          1             83             1   
top       7027      2018-08-07       FING             25          FING   
freq         1            4142       4326            172          4326   

            title          author           isbn  
count        4326            3857           3413  
unique       2355            1795           2020  
top     Cálculo :  Di Paolo, José  9789872923501  
freq           52              54             50  


In [13]:
lectura.validar_calidad(libros_total_koha) #validar la calidad de los datos

Validando calidad de datos del archivo...
Informacion de los datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4326 entries, 0 to 4325
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   barcode           4326 non-null   object 
 1   dateaccessioned   4326 non-null   object 
 2   booksellerid      0 non-null      float64
 3   homebranch        4326 non-null   object 
 4   price             103 non-null    float64
 5   replacementprice  1 non-null      float64
 6   itemcallnumber    4311 non-null   object 
 7   holdingbranch     4326 non-null   object 
 8   title             4326 non-null   object 
 9   author            3857 non-null   object 
 10  isbn              3413 non-null   object 
dtypes: float64(3), object(8)
memory usage: 371.9+ KB
Valores faltantes: 14271
Filas duplicadas: 0


* Se determina que las metricas necesarias para este analisis son "TItile, Author, ISBN"

Note como ahora si aparecen valores duplicados, esto se debe que antes la variable "barcode" era un unicode, es decir para cada observacion presentaba un id, esto termino siendo un problema ya que si hay filas duplicas pero por el barcode era imposible detectar.

Una vez detectado esos valores duplicados los eliminamos

In [14]:
libros_t2 = libros_total_koha[['title','author','isbn']]
lectura.validar_calidad(libros_t2) # Verificar calidad de los datos

Validando calidad de datos del archivo...
Informacion de los datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4326 entries, 0 to 4325
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   4326 non-null   object
 1   author  3857 non-null   object
 2   isbn    3413 non-null   object
dtypes: object(3)
memory usage: 101.5+ KB
Valores faltantes: 1382
Filas duplicadas: 1618


In [15]:
libros_t2 = libros_t2.drop_duplicates() # Eliminar duplicados

* Note que hay observaciones donde hay mas de un ISBN
Esto se debe a disntas razones, el primer isbn es de diez digitos y el segundo de trece o el segundo digito indica a la obra completa (o.c).

En ambos casos en conveniente quedarse con el primer ISBN presente.



In [16]:
filas_isbn_largo = libros_t2[libros_t2['isbn'].apply(lambda x: len(str(x)) > 13)]
print(filas_isbn_largo.head())


                                                 title           author  \
16                               Electrónica digital :              NaN   
35   Técnicas de prevención en la generación de sue...              NaN   
36   Técnicas de prevención en la generación de sue...              NaN   
225  Elementos de cálculo diferencial e integral: c...  Sadosky, Manuel   
233  Elementos de cálculo diferencial e integral: c...  Sadosky, Manuel   

                                     isbn  
16   847897735X | 9788478977352 (ISBN 13)  
35                8489650721 | 8489650802  
36                848965073X | 8489650802  
225               9505530013 | 9505530005  
233               9500530031 | 9500530007  


In [17]:
# Separar los ISBNs si están separados por espacio o "|", quedándonos con el primero
libros_t2['isbn'] = libros_t2['isbn'].str.split('|').str[0].str.strip()

* Ahora vamos a convertir los isbn de 10 digitos a 13. Esto se hace gracias a la funcion 'Convertir_isb' del modulo Conversor.

In [18]:
new_libros = conversor.convertir_isbn(libros_t2,'isbn') # Convertir los ISBNs a 13 dígitos


print(new_libros.head()) #Libros con los ISBN convertidos a 13 digitos

                                               title  \
0                                            Chaos :   
1                Analysis of observed chaotic data /   
2                             Proactive innovation :   
3  GMP qualification and validation of facilities...   
4  Introducción a la economía de la salud para in...   

                               author           isbn  
0               Alligood, Kathleen T.  9780387946771  
1                  Abarbanel, H. D. I  9780387983721  
2                Richter, Sergio Raúl  9798861825221  
3                                 NaN            NaN  
4  Martínez Licona, Fabiola Margarita  9786072813410  


*Un problema detectado es que hay observaciones donde presetan el mismo titulo y autor pero una observacion de ellas no tiene ISBN cargado, lo que significa que son datos iguales pero no se detectan. Decidi hacer una funcion que verifique si el titulo y autor coincide con otra observacion y esta tiene isbn, eliminara la que no tiene un valor en isbn

In [19]:
new_libros = conversor.eliminar_filas_sin_isbn(new_libros) # Eliminar filas sin ISBN pero con título y autor presentes en otras filas

A continuacion contrastaremos el df actual con el df "junto" estos se debe a que es posible de que nuestro df contenga folletos o articulos proveniente de congresos o similares. Estos no deben ser tomado en cuenta en nuestro analisis final

Para ello el csv 'junto' contiene el listado con los titulos de este tipo de categoria, lo utilizaremos para constrastar con el df raiz para dectectar y eliminar.

In [20]:
# Leer el archivo junto.csv
junto_df = pd.read_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/procesados/junto.csv') # Archivo con los datos concatenados para buscar coincidencias, si lo hay eliminarlos

# Filtrar el DataFrame new_libros para encontrar coincidencias
new_libros = new_libros[~new_libros['title'].isin(junto_df['title'])]

*NOTA.
*Hacer constraste con los titulos que SI van. seguramente no hay coincidencias
*sigo teniendo isbn faltante, ya considerar en eliminar
*Escribir el csv resultante de este archivo

## Analis sobre libros de consulta en Koha

In [21]:
consulta_koha = lectura.leer_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/consulta_koha.csv')

Archivo C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/consulta_koha.csv leído con éxito.


In [22]:
print(consulta_koha.head()) # Mostrar los primerosregistros

  barcode dateaccessioned  booksellerid homebranch    price  replacementprice  \
0    7232      2024-04-24           NaN       FING      NaN               NaN   
1    7233      2024-04-24           NaN       FING      NaN               NaN   
2    7228      2023-10-11           NaN       FING      NaN               NaN   
3    7224      2023-10-09           NaN       FING  18500.0               NaN   
4    7225      2023-10-09           NaN       FING  17500.0               NaN   

  itemcallnumber holdingbranch                                       title  \
0             32          FING  Chaotic evolution and strange attractors :   
1             32          FING              Chaotic and fractal dynamics :   
2             89          FING                        Científicas de acá :   
3             15          FING                                    Óptica /   
4             15          FING     Sears y Zemansky Física universitaria :   

           author               isbn  
0  Ru

In [23]:
print(consulta_koha.info()) # Información de los datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   barcode           1548 non-null   object 
 1   dateaccessioned   1548 non-null   object 
 2   booksellerid      0 non-null      float64
 3   homebranch        1548 non-null   object 
 4   price             159 non-null    float64
 5   replacementprice  2 non-null      float64
 6   itemcallnumber    1548 non-null   object 
 7   holdingbranch     1548 non-null   object 
 8   title             1548 non-null   object 
 9   author            1294 non-null   object 
 10  isbn              1194 non-null   object 
dtypes: float64(3), object(8)
memory usage: 133.2+ KB
None


In [24]:
# Mostrar un resumen descriptivo de las variables cualitativas
print(consulta_koha.describe(include='object'))

       barcode dateaccessioned homebranch itemcallnumber holdingbranch  \
count     1548            1548       1548           1548          1548   
unique    1548              81          1             69             1   
top       F519      2018-08-07       FING             99          FING   
freq         1            1368       1548             68          1548   

            title              author        isbn  
count        1548                1294        1194  
unique       1376                1037        1184  
top     Cálculo :  Spiegel, Murray R.  082183780X  
freq            9                  13           2  


In [25]:
lectura.validar_calidad(consulta_koha) #validar la calidad de los datos

Validando calidad de datos del archivo...
Informacion de los datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   barcode           1548 non-null   object 
 1   dateaccessioned   1548 non-null   object 
 2   booksellerid      0 non-null      float64
 3   homebranch        1548 non-null   object 
 4   price             159 non-null    float64
 5   replacementprice  2 non-null      float64
 6   itemcallnumber    1548 non-null   object 
 7   holdingbranch     1548 non-null   object 
 8   title             1548 non-null   object 
 9   author            1294 non-null   object 
 10  isbn              1194 non-null   object 
dtypes: float64(3), object(8)
memory usage: 133.2+ KB
Valores faltantes: 5091
Filas duplicadas: 0


In [26]:
libros_c = consulta_koha[['title','author','isbn']]
lectura.validar_calidad(libros_c) # Verificar calidad de los datos

Validando calidad de datos del archivo...
Informacion de los datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1548 non-null   object
 1   author  1294 non-null   object
 2   isbn    1194 non-null   object
dtypes: object(3)
memory usage: 36.4+ KB
Valores faltantes: 608
Filas duplicadas: 25


In [27]:
libros_c = libros_c.drop_duplicates() # Eliminar duplicados

In [28]:
# Separar los ISBNs si están separados por "|" o por espacio, quedándonos con el primero
libros_c['isbn'] = libros_c['isbn'].str.split(r'[| ]').str[0].str.strip()
print(libros_c.head())

                                        title          author           isbn
0  Chaotic evolution and strange attractors :  Ruelle, David.     0521368308
1              Chaotic and fractal dynamics :    Moon, F. C.,     0471545716
2                        Científicas de acá :             NaN  9786319002416
3                                    Óptica /   Hecht, Eugene  9788490354926
4     Sears y Zemansky Física universitaria :  Young, Hugh D.  9786073221900


In [29]:
libros_c2 = conversor.convertir_isbn(libros_c,'isbn') # Convertir los ISBNs a 13 dígitos


print(libros_c2.head()) #Libros con los ISBN convertidos a 13 digitos

                                        title          author           isbn
0  Chaotic evolution and strange attractors :  Ruelle, David.  9780521368308
1              Chaotic and fractal dynamics :    Moon, F. C.,  9780471545712
2                        Científicas de acá :             NaN  9786319002416
3                                    Óptica /   Hecht, Eugene  9788490354926
4     Sears y Zemansky Física universitaria :  Young, Hugh D.  9786073221900


In [30]:
libros_c2 = conversor.eliminar_filas_sin_isbn(libros_c2) # Eliminar filas sin ISBN pero con título y autor presentes en otras filas

In [31]:
# Filtrar el DataFrame new_libros para encontrar coincidencias
libros_c2 = libros_c2[~libros_c2['title'].isin(junto_df['title'])]

In [32]:
libros_c2.to_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/procesados/libros_consulta_koha.csv', index=False)

## Analisis sobre Archivo Bidi

In [33]:
bidi = lectura.leer_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/bidi.csv')

Archivo C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/bidi.csv leído con éxito.


In [34]:
bidi.columns = ['ID', 'Titulo', 'Autor', 'isbn', 'Editorial','Universidad','Numero_licencia','dato','Fecha_vencimiento_lic'] 
bidi


,ID,Titulo,Autor,isbn,Editorial,Universidad,Numero_licencia,dato,Fecha_vencimiento_lic
0,58930.0,La libertad como tecnología de gobierno. Derec...,Beatriz Dávilo [et al].,9789506984458,EDUNER,uner,20.0,14/3/2024 17:18,14/3/2025 17:18
1,58938.0,Semillas de orquídeas (de la región litoral),Víctor H. Lallana (compilador),9789506984793,EDUNER,uner,20.0,14/3/2024 17:18,14/3/2025 17:18
2,58925.0,Breve manual de lectura y escritura para la u...,Carolina Clerici,9789506984830,EDUNER,uner,20.0,14/3/2024 17:18,14/3/2025 17:18
3,58932.0,Manual de cálculos para el diseño de plantas d...,Elbio Miguel Woeffray,9789506984915,EDUNER,uner,20.0,14/3/2024 17:18,14/3/2025 17:18
4,58937.0,Revolución y lucha por la organización. Primer...,Juan A. Vilar,9789506984328,EDUNER,uner,20.0,14/3/2024 17:18,14/3/2025 17:18
...,...,...,...,...,...,...,...,...,...
6098,62076.0,América en el pensamiento de Alfonso Reyes,Alfonso Reyes,9786071638182,Fondo de Cultura Económica,uner,100.0,30/8/2024 18:03,30/8/2025 18:03
6099,62077.0,El enamoramiento y el mal de amores,Alberto Orlandini,9786071645197,Fondo de Cultura Económica,uner,100.0,30/8/2024 18:03,30/8/2025 18:03
6100,62078.0,Cáncer: herencia y ambiente,Cristina Cortinas,9786071644930,Fondo de Cultura Económica,uner,100.0,30/8/2024 18:04,30/8/2025 18:04
6101,62079.0,Venenos,Juan Luis Cifuentes,9786071644923,Fondo de Cultura Económica,uner,100.0,30/8/2024 18:04,30/8/2025 18:04


In [35]:
lectura.validar_calidad(bidi) # Validar la calidad de los datos

Validando calidad de datos del archivo...
Informacion de los datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6103 entries, 0 to 6102
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     2627 non-null   float64
 1   Titulo                 6103 non-null   object 
 2   Autor                  6101 non-null   object 
 3   isbn                   6103 non-null   int64  
 4   Editorial              6101 non-null   object 
 5   Universidad            6103 non-null   object 
 6   Numero_licencia        2627 non-null   float64
 7   dato                   2627 non-null   object 
 8   Fecha_vencimiento_lic  2627 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 429.2+ KB
Valores faltantes: 13908
Filas duplicadas: 0


In [36]:
bidi2 = bidi[['Titulo','Autor','isbn']]
lectura.validar_calidad(bidi2) # Verificar calidad de los datos

Validando calidad de datos del archivo...
Informacion de los datos
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6103 entries, 0 to 6102
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Titulo  6103 non-null   object
 1   Autor   6101 non-null   object
 2   isbn    6103 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 143.2+ KB
Valores faltantes: 2
Filas duplicadas: 0


In [37]:
bidi2

,Titulo,Autor,isbn
0,La libertad como tecnología de gobierno. Derec...,Beatriz Dávilo [et al].,9789506984458
1,Semillas de orquídeas (de la región litoral),Víctor H. Lallana (compilador),9789506984793
2,Breve manual de lectura y escritura para la u...,Carolina Clerici,9789506984830
3,Manual de cálculos para el diseño de plantas d...,Elbio Miguel Woeffray,9789506984915
4,Revolución y lucha por la organización. Primer...,Juan A. Vilar,9789506984328
...,...,...,...
6098,América en el pensamiento de Alfonso Reyes,Alfonso Reyes,9786071638182
6099,El enamoramiento y el mal de amores,Alberto Orlandini,9786071645197
6100,Cáncer: herencia y ambiente,Cristina Cortinas,9786071644930
6101,Venenos,Juan Luis Cifuentes,9786071644923


In [38]:
#bidi2.to_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/procesados/libros_bidi.csv', index=False) # Guardar el archivo en la carpeta procesados